In [2]:
import numpy as np
import _pickle as cPickle

In [23]:
def l0_check(a: np.array):
    b = np.array(a)
    for i in range(len(b)):
        if (b[i] != 0):
            b[i] = 1
    return b

In [24]:
FILE_FOR_MODEL = "05_class_Logistic_regression_model.txt"
class LogisticRegression:
    def __init__(self, alpha: float, l0: float, l1: float, l2: float, 
                 stop_iter: float, stop_delta: float, verbose: bool, model: object):
        '''
        Linerar Regression object constructor
        
        :param alpha: alpha param for gradient descent
        :param l0: L0 regularization coefficient
        :param l1: L1 regularization coefficient
        :param l2: L2 regularization coefficient
        :param stop_iter: maximum iterations of traing
        :param stop_delta: stop iteration delta
        :param verbose: show verbose information
        :param model: get params from other model
        '''
        if (model == None):
            self.alpha = alpha
            self.l0 = l0
            self.l1 = l1
            self.l2 = l2
            self.stop_iter = stop_iter
            self.stop_delta = stop_delta
            self.verbose = verbose
            self.model = model
        else:            
            self.alpha = model['alpha']
            self.l0 = model['l0']
            self.l1 = model['l1']
            self.l2 = model['l2']
            self.stop_iter = model['stop_iter']
            self.stop_delta = model['stop_delta']
            self.verbose = model['verbose']
            self.model = model['model']
    
    def train(self, X: np.array, y: np.array, warm_start: bool):
        '''
        Fit Linear Regression params
        
        :param X: training data
        :param y: training ansewers
        :param warm_start: must be set True to continue training, false to reset params
        '''
        
        if (warm_start == False):
            self.w = np.random.rand(len(X[0]))
        else:
            file = open(FILE_FOR_MODEL, 'rb')
            self.w = cPickle.load(file)
            file.close()
            
        indicator_for_cycle_exit_criterion = 1   
        
        for j in range(int(self.stop_iter)):
            
            grad = X.T.dot(y-1/(1+np.exp(X.dot(self.w.T))))/len(y) + self.l0*l0_check(self.w) + self.l1*np.sign(self.w) + self.l2*2*self.w
            self.w = self.w - self.alpha*grad
            j += 1
            
            if (self.verbose == True):
                if (j % 100000 == 0):
                    print('iteration = ', j, 'vect =', self.w, 'grad =', grad)
            
            if(np.linalg.norm(grad) < self.stop_delta):
                indicator_for_cycle_exit_criterion = 0   
                print('|grad| < stop_delta')
                break
                
        if (indicator_for_cycle_exit_criterion):
            print(self.stop_iter, 'iterations done')
        
        file = open(FILE_FOR_MODEL, 'wb')
        cPickle.dump(self.w, file)
        file.close()
        print('train results saved in', FILE_FOR_MODEL)
            
    
    def get_params(self) -> dict:
        '''
        Return model params
        
        :return: dict of model params
        '''
        dict_params = {}
        dict_params['alpha'] = self.alpha
        dict_params['l0'] = self.l0
        dict_params['l1'] = self.l1
        dict_params['l2'] = self.l2
        dict_params['stop_iter'] = self.stop_iter
        dict_params['stop_delta'] = self.stop_delta
        dict_params['verbose'] = self.verbose
        dict_params['model'] = self.model
        return dict_params
    
    def predict(self, X: np.array) -> np.array:
        file = open(FILE_FOR_MODEL, 'rb')
        self.w = cPickle.load(file)
        file.close()
        '''
        Predit answers on given data
        
        :param X: data
        :return: predicted answers
        '''
        return 1/(1+np.exp(X.dot(self.w.T)))
    
    def test(self, X: np.array, y: np.array, metric=None) -> float:
        file = open(FILE_FOR_MODEL, 'rb')
        self.w = cPickle.load(file)
        file.close()
        '''
        Test the model
        
        :param X: test data
        :param y: test answers
        :param metric: must be a function of 2 numpy arrays. If None, MSE is used.
        :return: metric value
        '''
        return (1/(1+np.exp(X.dot(self.w.T))) - y).dot((1/(1+np.exp(X.dot(self.w.T))) - y).T)/len(y)

In [31]:
logr = LogisticRegression(alpha=1e-4, l0=0., l1=1., l2=0., stop_iter=1e6, stop_delta=1e-4, verbose=False, model=None)

In [16]:
x_log = np.array([[1,-100],
                    [1,-80],
                    [1,-50],
                    [1,-60],
                    [1,-40],
                    [1,20],
                    [1,60],
                    [1,90],
                    [1,70],
                    [1,80],
                    [1,-70],
                    [1,100]])
y_log = np.array([0,0,0,0, 0,0,1,1,1,1,0,1])

In [17]:
logr.verbose = True
logr.train(X=x_log, y=y_log, warm_start=False)

iteration =  100000 vect = [ 1.20862322 -0.06592539] grad = [-0.03727316  0.00052396]
iteration =  200000 vect = [ 1.547615   -0.07111928] grad = [-0.03099486  0.00050984]
iteration =  300000 vect = [ 1.83531546 -0.07607385] grad = [-0.02679134  0.00048016]
iteration =  400000 vect = [ 2.0871894  -0.08071733] grad = [-0.02372705  0.00044867]
iteration =  500000 vect = [ 2.31214622 -0.08505351] grad = [-0.02135627  0.00041897]
iteration =  600000 vect = [ 2.5158367  -0.08910555] grad = [-0.01944522  0.00039189]
iteration =  700000 vect = [ 2.70213027 -0.09290013] grad = [-0.01785973  0.00036746]
iteration =  800000 vect = [ 2.87383598 -0.09646282] grad = [-0.01651659  0.00034547]
iteration =  900000 vect = [ 3.0330845  -0.09981672] grad = [-0.01536075  0.00032566]
iteration =  1000000 vect = [ 3.18154624 -0.10298234] grad = [-0.01435385  0.00030777]
1000000.0 iterations done
train results saved in 05_class_Logistic_regression_model.txt


In [32]:
logr.verbose = True
logr.train(X=x_log, y=y_log, warm_start=False)

iteration =  100000 vect = [  9.63816342e-05  -4.80415475e-02] grad = [ -1.07674242e+00   4.42298340e-06]
iteration =  200000 vect = [ -8.92480281e-05  -4.80415470e-02] grad = [  9.23258330e-01   7.24484908e-07]
iteration =  300000 vect = [ -7.49064816e-05  -4.80415510e-02] grad = [  9.23259076e-01  -3.58480715e-06]
iteration =  400000 vect = [ -6.05901599e-05  -4.80415542e-02] grad = [  9.23259820e-01  -7.77565912e-06]
iteration =  500000 vect = [ -4.62377750e-05  -4.80415565e-02] grad = [  9.23260565e-01  -1.18512082e-05]
iteration =  600000 vect = [ -3.18679214e-05  -4.80415434e-02] grad = [  9.23261308e-01  -1.37741597e-05]
iteration =  700000 vect = [ -1.75418416e-05  -4.80415366e-02] grad = [  9.23262050e-01  -1.65705523e-05]
iteration =  800000 vect = [ -3.22339854e-06  -4.80415345e-02] grad = [  9.23262793e-01  -2.00238278e-05]
iteration =  900000 vect = [  1.11288108e-05  -4.80415331e-02] grad = [  9.23263537e-01  -2.35824302e-05]
iteration =  1000000 vect = [  2.55022943e-05 

In [79]:
logr.get_params()

{'alpha': 0.0001,
 'l0': 0.0,
 'l1': 0.0,
 'l2': 0.0,
 'model': None,
 'stop_delta': 0.01,
 'stop_iter': 1000000.0,
 'verbose': True}

In [80]:
logr2=LogisticRegression(**logr.get_params())

In [81]:
x2_log = np.array([[1,-121],
                    [1,-81],
                    [1,-58],
                    [1,-69],
                    [1,-35],
                    [1,25],
                    [1,76],
                    [1,87],
                    [1,66],
                    [1,60],
                    [1,-70],
                    [1,100]])
y2_log = np.array([0,0,0,0, 0,0,1,1,1,1,0,1])

In [82]:
logr2.train(X=x_log, y=y_log, warm_start=False)

100000
iteration =  100000 vect = [ 1.21215986 -0.06597516] grad = [-0.03719863  0.000524  ]


KeyboardInterrupt: 

In [84]:
logr2.train(X=x_log, y=y_log, warm_start=True)

300000
400000
iteration =  400000 vect = [ 2.08944178 -0.08075993] grad = [-0.02370176  0.00044838]
500000
iteration =  500000 vect = [ 2.31417367 -0.08509329] grad = [-0.02133623  0.0004187 ]
600000
iteration =  600000 vect = [ 2.51768282 -0.08914276] grad = [-0.01942879  0.00039164]
700000
iteration =  700000 vect = [ 2.70382593 -0.09293502] grad = [-0.01784594  0.00036724]
800000
iteration =  800000 vect = [ 2.87540415 -0.09649562] grad = [-0.0165048   0.00034527]
900000
iteration =  900000 vect = [ 3.03454297 -0.09984765] grad = [-0.01535054  0.00032548]
1000000
iteration =  1000000 vect = [ 3.18290913 -0.10301157] grad = [-0.0143449   0.00030761]
1000000.0 iterations done


In [85]:
logr2.test(X=x2_log, y=y2_log)

0.010614571805499589